In [26]:
# Nobuaki Fuji @ipgp September 2025
using Pkg, BenchmarkTools


cd(@__DIR__)
Pkg.activate("../..")

# below are the tools to debug the code
using Revise # this is very cool since 
using Profile, StatProfilerHTML


include("../src/imageReader.jl") # read 2D images for models

include("../src/OPTwrappers.jl") 


  Activating project at `~/Documents/Github/flexibleDSM`


quasiNumericalOperatorConstruction (generic function with 1 method)

In [2]:
iExperiment = 1

# 'iExperiment' can be fixed to be 1 if the user does not need to perform benchmark tests
# (which should be the case for the most of the time when the software gets stabilised)

1

In [3]:

#region Choose backend depending on environment
if isdefined(Main, :IJulia) || get(ENV, "JULIA_EDITOR", "") == "code"
    # Use CairoMakie for inline if in notebook or VS Code
    using CairoMakie
else
    using GLMakie
end

In [4]:
famousEquationType="2DacousticTime"
exprs,fields,vars,extexprs,extfields,extvars,coordinates,∂,∂²=famousEquations(famousEquationType)

(Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), u(x, y, t), v(x, y), f(x, y, t), f(x, y, t), 1, (x, y, t), Any[Differential(x), Differential(y), Differential(t)], Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])

In [5]:
modelName="marmousi"

modelDefinitionMethod="2DimageFile" # ToyModel or 2DimageFile (or 1DsphericalPlanet)
model =nothing


if modelDefinitionMethod !== nothing
        
    #region Model input - option i) Model domain definition

    if modelDefinitionMethod === "ToyModel"
        DomainWindow=(DomainWindowT=1.0,DomainWindowX=1.0,DomainWindowY=1.0,DomainWindowZ=1.0)
        ModelSizeTXYZ=(ModelSizeT=101,ModelSizeX=101,ModelSizeY=101,ModelSizeZ=0)
    end

    #endregion

    #region Model input - option ii) Read a file (2D or 3D) and define Δs

    if modelDefinitionMethod === "2DimageFile"

        #imagefile="../data/model/random/colourful.jpg"
        #imagefile="../data/model/artemis/IMG_6098.jpeg"
        #imagefile="../data/model/random/tmp.png"
        imagefile = "../data/model/random/marmousi.png"
        colormap = "jet" #colormap can be RGB vector or predefined colormap

        #model=read2DimageModel(imagefile,colormap;Nwidth=10,Nheight=10,showRecoveredImage=false)
        model=read2DimageModel(imagefile,colormap;showRecoveredImage=false)
    end
    #endregion

    #region Model input - option iii) Read a file (1D spherical planet models)

    if modelDefinitionMethod ==="1DsphericalPlanet"
        # use some programmes that are developed during Xmas 2023
        # inputModels.jl
    end

#endregion
end

In [6]:
coordinates

(x, y, t)

In [7]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as coordinates 



IneedExternalSources = true
maskedRegionForSourcesInSpace = nothing

#DrWatson configurations

orderBtime=1
orderBspace=1
pointsInSpace=2
pointsInTime=2

WorderBspace=1
WorderBtime=1
supplementaryOrder=2

2

In [8]:
models=[] # you might need to make this empty tuple first, otherwise one-member tuple can be misinterpreted
models=push!(models, (model .* 0.2 .+ 0.4))

1-element Vector{Any}:
 [0.4 0.4 … 0.4 0.4; 0.4 0.4 … 0.4 0.4; … ; 0.5138386099547898 0.5163891573813251 … 0.5501948972360029 0.5472638856339349; 0.5300777283024765 0.5308589172720881 … 0.5545259469249547 0.5545259469249547]

In [9]:
fakeNt = 1
timeMarching = any(a -> a === timeDimensionString, string.(coordinates)) 
if timeMarching
    fakeNt = pointsInTime+1
    modelPoints = (size(model)...,fakeNt) # Nx, Ny etc thing. Nt is also mentioned and it should be the last element!
else
    modelPoints = (size(model))
end

(676, 802, 3)

In [10]:
fakeNt

3

In [11]:
maskedRegionForSourcesInSpace  = Array{CartesianIndex,1}(undef,0) # it is important to decalre the type of this
maskedRegionForSourcesInSpace = push!(maskedRegionForSourcesInSpace, CartesianIndex(modelPoints[1:end-1].÷2))# in Ndimension (or Ndimension  - 1 if timeMarching)
# in this example, I put a point source at the centre of the model space

forceModels =((1.0)) # if your model does not have anything special material parameters then it's how it's written

concreteModelParameters = @strdict famousEquationType Δnum orderBtime orderBspace WorderBtime WorderBspace supplementaryOrder pointsInSpace pointsInTime IneedExternalSources modelName models modelPoints forceModels maskedRegionForSourcesInSpace iExperiment

Dict{String, Any} with 16 entries:
  "IneedExternalSources"          => true
  "models"                        => Any[[0.4 0.4 … 0.4 0.4; 0.4 0.4 … 0.4 0.4;…
  "maskedRegionForSourcesInSpace" => CartesianIndex[CartesianIndex(338, 401)]
  "iExperiment"                   => 1
  "orderBtime"                    => 1
  "Δnum"                          => (1.0, 1.0, 1.0)
  "pointsInSpace"                 => 2
  "WorderBspace"                  => 1
  "pointsInTime"                  => 2
  "WorderBtime"                   => 1
  "supplementaryOrder"            => 2
  "orderBspace"                   => 1
  "modelPoints"                   => (676, 802, 3)
  "forceModels"                   => 1.0
  "modelName"                     => "marmousi"
  "famousEquationType"            => "2DacousticTime"

In [12]:
safeget(A, inds...; default=0) = checkbounds(Bool, A, inds...) ? A[inds...] : default

safeget (generic function with 1 method)

In [ ]:
@profilehtml opt = myProduceOrLoad(makeCompleteCostFunctions,concreteModelParameters,"numOperators","quasiNum")


┌ Info: File /Users/nobuaki/Documents/Github/flexibleDSM/data/numOperators/quasiNum_f52922c2.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/dUlaU/src/saving_files.jl:106
┌ Info: File /Users/nobuaki/Documents/Github/flexibleDSM/data/semiSymbolics/semiSymbolics_f7d7f762.jld2 does not exist. Producing it now...
└ @ DrWatson /Users/nobuaki/.julia/packages/DrWatson/dUlaU/src/saving_files.jl:106


(bigα, varM) = (Any[Any[(node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(3, 1, 1)), (node = -(v(x, y)^2), nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 3, 1)), (node = 1, nᶜ = CartesianIndex(1, 1, 1), n = CartesianIndex(1, 1, 3))];;], Any[v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉ v₁ v₂ v₃ v₄ v₅ v₆ v₇ v₈ v₉])
modifiedμ[iCoord] = output["modμ"] = 

In [23]:
statprofilehtml()

┌ Warning: The profile data buffer is full; profiling probably terminated
│ before your program finished. To profile for longer runs, call
│ `Profile.init()` with a larger buffer and/or larger delay.
└ @ Profile /Users/nobuaki/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Profile/src/Profile.jl:641
┌ Info: Wrote profiling output to file:///Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html .
└ @ StatProfilerHTML /Users/nobuaki/.julia/packages/StatProfilerHTML/WyFko/src/StatProfilerHTML.jl:27


In [24]:
using InteractiveUtils
run(`open /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html`)  

Process(`open /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html`, ProcessExited(0))

In [20]:
function buggy_function()
    s = 0
    for i in 1:10^7
        s += sin(i)
        if i == 5_000_000
            error("boom!")
        end
    end
    return s
end

try
    @profilehtml buggy_function()
catch err
    @warn "Crashed" err
end

# Show the flamegraph in your browser
statprofilehtml()

┌ Warning: Crashed
│   err = ErrorException("boom!")
└ @ Main /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:15
┌ Info: Wrote profiling output to file:///Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html .
└ @ StatProfilerHTML /Users/nobuaki/.julia/packages/StatProfilerHTML/WyFko/src/StatProfilerHTML.jl:27


In [21]:
using InteractiveUtils
run(`open /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html`)  

Process(`open /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/myOwnApplications/statprof/index.html`, ProcessExited(0))